In [8]:
from huggingface_hub import notebook_login
notebook_login()

Assign a binary label to high engagement and low engagement videos

In [29]:
import pandas as pd

df = pd.read_csv("../data/youtube_data.csv")  

views_threshold = 1000000
likes_threshold = 100000

df["label"] = ((df["Views"] > views_threshold) & (df["Likes"] > likes_threshold)).astype(int)

df = df.dropna(subset=["Title", "Views", "Likes"])

df.to_csv("../data/titles_dataset.csv", index=False)

print(df["label"])

0    1
1    1
2    1
3    1
4    1
5    0
6    1
7    0
8    0
9    0
Name: label, dtype: int64


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

# 1. Load your labeled dataset
df = pd.read_csv("../data/titles_dataset.csv")  # make sure this file exists

# Balance classes or inspect class distribution
# print(df['label'].value_counts())

# 2. Split into train/test
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"])

# 3. Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 4. Load tokenizer & model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 5. Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["Title"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

# 6. Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 7. Ensure the labels are included in the dataset and in the correct format for PyTorch
def format_dataset(dataset):
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
    return dataset

train_dataset = format_dataset(train_dataset)
test_dataset = format_dataset(test_dataset)

# 8. Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./bert_youtube_classifier",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# 9. Define metrics
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 10. Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# 11. Train the model
trainer.train()

# 12. Evaluate
trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

/Users/arnoldm./NLP/venv2/lib/python3.11/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/6c/z4wf1j4d35d6_2zwzxrv9mgh0000gn/T/ipykernel_15250/2781558848.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.702538,0.500000,0.666667,0.500000,1.000000
2,No log,0.686515,0.500000,0.666667,0.500000,1.000000
3,No log,0.686286,0.500000,0.666667,0.500000,1.000000


{'eval_loss': 0.7025378942489624,
 'eval_accuracy': 0.5,
 'eval_f1': 0.6666666666666666,
 'eval_precision': 0.5,
 'eval_recall': 1.0,
 'eval_runtime': 0.9207,
 'eval_samples_per_second': 2.172,
 'eval_steps_per_second': 1.086,
 'epoch': 3.0}